In [1]:
import pandas as pd
import re

In [2]:
ls

sample_data/  total_concat.csv


In [23]:
# csv 불러오기
total = pd.read_csv('total_concat.csv')

<ipython-input-23-fd5f24435501>:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  total = pd.read_csv('total_concat.csv')


In [24]:
# 중복된 띄어쓰기,줄바꿈을 띄어쓰기 1개로 전체 바꾸기
total['content'] = total['content'].str.replace('\s+', ' ')

<ipython-input-24-f008cf67a116>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  total['content'] = total['content'].str.replace('\s+', ' ')


In [25]:
#이모지 & 특수기호 제거
total['content'] = total['content'].str.replace(pat = r'[^가-힣A-Za-z0-9.-/:\']', repl = r' ', regex = True)

In [26]:
def clean_text(inputString):
  text_rmv = re.sub('ه ه [-=+,#\?^*\"※`~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]ܤه  ه ', ' ', inputString)
  text_rmv = ' '.join(text_rmv.split())
  return text_rmv

# 액센트 제거

def remove_accents(raw_text):
    raw_text = re.sub(u"[àáâãäåạ]", 'a', raw_text)
    raw_text = re.sub(u"[èéêë]", 'e', raw_text)
    raw_text = re.sub(u"[ìíîï]", 'i', raw_text)
    raw_text = re.sub(u"[òóôõö]", 'o', raw_text)
    raw_text = re.sub(u"[ùúûü]", 'u', raw_text)
    raw_text = re.sub(u"[ýÿ]", 'y', raw_text)
    raw_text = re.sub(u"[ß]", 'ss', raw_text)
    raw_text = re.sub(u"[ñ]", 'n', raw_text)
    raw_text = re.sub(u"[➍④]", '4', raw_text)
    raw_text = re.sub(u"𝒂𝒕𝒐", 'ato', raw_text)
    raw_text = re.sub(u"Аҳŗịṅḡ", 'axring', raw_text)
    raw_text = re.sub(u"аҳŗịṅḡ", 'axring', raw_text)
    return raw_text

In [27]:
for i in range(len(total['content'])):
    a = clean_text(total['content'][i])
    a = remove_accents(a)
    total['content'][i] = a

<ipython-input-27-a0770f4b8b7c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total['content'][i] = a


In [28]:
# 추가 전처리
#punct = "'?!,#$%\'()*+-/;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', 'o':''} 
searchfilter_mapping = {"ㅌ ㄹ ㄱ ㄹ": "텔레그램", "ㅌ ㄹ ㄱ ㄹ": "텔레그램","ㅌㄹ": "텔레", "ㅌ ㄹ": "텔레", "ㅌㄹㄱㄹ": "텔레그램", "텔 레": "텔레", "그 램":"그램", "텔레 그램":"텔레그램","문 의":"문의","ka톡":"카톡", "ka 톡": "카톡", "ｋa톡":"카톡", "카 카 오 톡":"카카오톡", "gt":"", "lt":"", "p i n k m a n 1 1 4": "pinkman114", "F a s t i c e119": "Fastice119"}

def clean(text, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    # for p in punct:
    #     text = text.replace(p, f' {p} ')

    # for p in punct:
    #     text = text.replace(p, f' {p} ') 
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': '', 'ه ':'', '∞': ''}
    for s in specials:
        text = text.replace(s, specials[s])

    return text.strip()

def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(r'http\S+', repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = re.compile(r"(?<=\b\w)\s(?=\w\b)") # 띄어쓰기 된  아이디 결합
    text = re.sub(pattern=pattern, repl='', string=text)
    #pattern = '[^\w\s\n]'         # 특수기호제거
    #text = re.sub(pattern=pattern, repl='', string=text)
    #text = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', string=text)
    #text = re.sub('\n', '.', string=text)
    return text 

In [29]:
for i in range(len(total['content'])):
    a = clean(total['content'][i], punct_mapping)
    a = clean(total['content'][i], searchfilter_mapping)
    a = clean_str(a)
    total['content'][i] = a

<ipython-input-29-bde88ab5dd81>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total['content'][i] = a


In [30]:
# 중복된 띄어쓰기,줄바꿈을 띄어쓰기 1개로 전체 바꾸기
total['content'] = total['content'].str.replace('\s+', ' ')

<ipython-input-30-f008cf67a116>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  total['content'] = total['content'].str.replace('\s+', ' ')


In [31]:
for text in range(len(total['content'])):
  print(total['content'][text])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
여러분에 안식처 아이스 하이븐 텔레 문의 ice haven 술사는곳 떨파는곳소개 후리하는방법 작대기퀄리티 아이스사용방법 떨구매 엑시 떨떨 케이 샘플20만 빙두 사끼 아이스뽕 아이스드랍 사끼 빙두 
여러분에 안식처 아이스 하이븐 텔레 문의 ice haven 술사는곳 떨파는곳소개 후리하는방법 작대기퀄리티 아이스사용방법 떨구매 엑시 떨떨 케이 샘플20만 빙두 사끼 아이스뽕 아이스드랍 사끼 빙두 
청소다 뭐다 바쁜 미진도 집에 작대기술 아이스작대기 
청소다 뭐다 바쁜 미진도 집에 작대기술 아이스작대기 
Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 마약효과 빠른구매 정품아이스파는곳 아이스판매처 크리스탈아이스 클럽약 케이 인디카 엑스터시 떨 아이스팝니다 허브 허브액상 작대기아이스 아이스작대기 
Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 마약효과 빠른구매 정품아이스파는곳 아이스판매처 크리스탈아이스 클럽약 케이 인디카 엑스터시 떨 아이스팝니다 허브 허브액상 작대기아이스 아이스작대기 
Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 마약효과 빠른구매 정품아이스파는곳 아이스판매처 크리스탈아이스 클럽약 케이 인디카 엑스터시 떨 아이스팝니다 허브 허브액상 작대기아이스 아이스작대기 
여러분에 안식처 아이스 하이븐 텔레 문의 ice haven 작대기주사 아이스효능직빵 작대기퀄리티 아이스작대기 떨후기 엑시 떨팝니다 떨가격 떨팝니다 허브 작대기 아이스 작대기술 샘플20만 사끼 빙두 
여러분에 안식처 아이스 하이븐 텔레 문의 ice haven 작대기주사 아이스효능직빵 작대기퀄리티 아이스작대기 떨후기 엑시 떨팝니다 떨가격 떨팝니다 허브 작대기 아이스 작대기술 샘플20만 사끼 빙두 
여러분에 안식처 아이스 하이븐 텔레 문의 ice haven 작대기주사 아이스효능직빵 작대기퀄리티 아이스작대기 떨후기 엑시 떨팝니다 떨가격 떨팝니다 허브 작대기 아이스 작대기술 샘플20만 사끼 빙두 
여러분에 안식처 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 천안아이스 서울아이스 떨판매 인잭하는법 클럽캔디 울산아이스 떨 최음제 캔디 클럽파티 엑스터시 엘에디 허브 아이스뽕 엘에디 아이스팝니다 대마 작대기삽니다 
Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 천안아이스 서울아이스 떨판매 인잭하는법 클럽캔디 울산아이스 떨 최음제 캔디 클럽파티 엑스터시 엘에디 허브 아이스뽕 엘에디 아이스팝니다 대마 작대기삽니다 
Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 천안아이스 서울아이스 떨판매 인잭하는법 클럽캔디 울산아이스 떨 최음제 캔디 클럽파티 엑스터시 엘에디 허브 아이스뽕 엘에디 아이스팝니다 대마 작대기삽니다 
텔레그램 id pinkman114 엑시터시구입하는방법 밀크케타민구입방법 허브사는법 케타민원석 대마구매방법 엑시터시 떨 
텔레그램 id pinkman114 포도허브 고퀄아이스사는법 울산드랍퍼모집 대전아이스 사끼 전남마약 몰리 
텔레그램 id pinkman114 포도허브 고퀄아이스사는법 울산드랍퍼모집 대전아이스 사끼 전남마약 몰리 
텔레그램 id pinkman114 전라도도리도리 MDMA 수면유도제 LSD 머쉬룸 고퀄아이스구입하는법 경남안전딜러 
텔레 문의 Happy bbong 해피 해피뽕 아이스작대기 전문 딜러 술파는곳 얼음사는곳얼음파는곳 술구매 인슐린주사기 떨삽니다 떨구매 엑시 떨구매 떨파는곳 아이스 아이스팝니다 허브 아이스 차가운술 케타민 
. 잠깐만 기다려.금방나갈게 아이스작대기 차가운술 p m9 
텔레그램 id pinkman114 합성떨 대전파퍼 떨액파는곳 강북발정제 펜타닐 밀크케이하는법 차가운술 
텔레그램 id pinkman114 창원드랍퍼구인 하이드로몰핀구입하는법 합성대마초 모르핀구매처 강서액상대마 원석케타민 수면유도제사는곳 
텔레그램 id pinkman114 하우스파티 마약 뽕구하는법 크리스탈 액상대마 강남케미컬약 인슐린하는방법 
텔레그램 id pinkman114 

In [32]:
total.to_csv('total_preprocessed.csv', index = False)